In [59]:
import pymysql
import pandas as pd
connection1 = pymysql.connect(host = '119.74.24.181', user = 'htx', password = 'Police123456', database = 'ASTRO')
connection2 = pymysql.connect(host = 'localhost', user = 'root', password = 'X-rayisharmful01', database = 'sys')

df = pd.read_sql_query("SELECT * FROM astro.scam_management_system", connection1)

df_test = pd.read_sql_query("SELECT * FROM sys.sys_config", connection2)




C:\Users\user\AppData\Local\Temp\ipykernel_9596\3561170033.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM astro.scam_management_system", connection1)
C:\Users\user\AppData\Local\Temp\ipykernel_9596\3561170033.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql_query("SELECT * FROM sys.sys_config", connection2)


In [69]:
import pymysql
import pandas as pd
connection2 = pymysql.connect(host = 'localhost', user = 'root', password = 'X-rayisharmful01', database = 'sys')

df_test = pd.read_sql_query("SELECT * FROM sys.sys_config", connection2)


df_test2 = df_test['variable'].value_counts().reset_index()

# rename columns
df_test2.columns = ['variable', 'count']
x_axis = df_test2['variable'].tolist()
y_axis = df_test2['count'].tolist()


C:\Users\user\AppData\Local\Temp\ipykernel_9596\1866772294.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql_query("SELECT * FROM sys.sys_config", connection2)


['diagnostics.allow_i_s_tables',
 'diagnostics.include_raw',
 'ps_thread_trx_info.max_length',
 'statement_performance_analyzer.limit',
 'statement_performance_analyzer.view',
 'statement_truncate_len',
 'test',
 'testing']

In [57]:
telco_count_series = telco_df.groupby('telco').size()
telco_name_list = telco_count_series.index.to_list()
telco_count_list = telco_count_series.to_list()
telco_count_series[0] = 37
telco_count_series

telco
M1            37
MyRepublic    84
Singtel       25
Starhub        7
dtype: int64

In [51]:
df_scam_type_interim = df[['scam_type', 'amount_scammed']].copy()
df_scam_type_interim['scam_type'] = df_scam_type_interim['scam_type'].str.lower()
df_scam_type_interim['scam_type'] = df_scam_type_interim['scam_type'].replace('loan scan', 'loan scam')

df_scam_type_interim = df_scam_type_interim.groupby('scam_type').agg({'scam_type': 'count', 'amount_scammed': 'sum'})
df_scam_type_interim.columns = ['num_reports', 'total_amount_scammed']
df_scam_type_interim = df_scam_type_interim.rename_axis('scam_type').reset_index()
df_scam_type_interim['scam_type'] = df_scam_type_interim['scam_type'].str.title()

df_scam_type_interim = df_scam_type_interim.sort_values(by = 'num_reports', ascending = False)
top_scam_type_list = df_scam_type_interim.iloc[0:10]['scam_type'].values.tolist()

print(top_scam_type_list)
#   Data Cleaning
df_scam_type = df[['date_assigned', 'scam_type']].copy()
df_scam_type = df_scam_type.dropna(subset=['date_assigned', 'scam_type'])
df_scam_type['scam_type'] = df_scam_type['scam_type'].str.lower()
df_scam_type['scam_type'] = df_scam_type['scam_type'].replace('loan scan', 'loan scam')
df_scam_type['scam_type'] = df_scam_type['scam_type'].str.title()

df_scam_type = df_scam_type[df_scam_type["scam_type"].isin(top_scam_type_list)].reset_index()


df_scam_type['date_assigned'] = pd.to_datetime(df_scam_type['date_assigned'])
counts = df_scam_type.groupby(['scam_type', 'date_assigned']).size()
df_scam_type = counts.reset_index(name='count')

pd.set_option('display.max_rows', 500)


['Job Scam', 'Investment Scam', 'E-Commerce Scam', 'Friend Impersonation Scam', 'Other Scam', 'Rental Scam', 'Gois', 'Loan Scam', 'Love/Parcel Scam', 'S/M Impersonation Scam']


In [17]:

from datetime import timedelta, timezone
df_recovery_trend = df[['date_assigned', 'amount_scammed', 'latest_balance_seized']].copy()
df_recovery_trend = df_recovery_trend.fillna(0)

iteration_count = len(df_recovery_trend.index)
dates = []
amount_recovered_list = []
amount_recovered_interim = 0
start_date = df_recovery_trend['date_assigned'][0]
start_date = round(start_date.replace(tzinfo=timezone.utc).timestamp()) * 1000
current_date = start_date
end_date = df_recovery_trend['date_assigned'][iteration_count-1]

while current_date != end_date:
    dates.append(current_date)
    current_date += timedelta(days = 1)
dates.append(current_date)

current_date = start_date

for i in range(iteration_count):
    if df_recovery_trend['date_assigned'][i] == current_date:
        if df_recovery_trend['latest_balance_seized'][i] >= df_recovery_trend['amount_scammed'][i]:
            amount_recovered_interim += df_recovery_trend['amount_scammed'][i]
        else:
            amount_recovered_interim += df_recovery_trend['latest_balance_seized'][i]

    else:
        amount_recovered_interim = round(amount_recovered_interim, 2)
        amount_recovered_list.append(amount_recovered_interim)
        amount_recovered_interim = 0
        current_date += timedelta(days = 1)
        
        while df_recovery_trend['date_assigned'][i] != current_date:
            amount_recovered_list.append(0)   
            current_date += timedelta(days = 1)
            
        if df_recovery_trend['latest_balance_seized'][i] >= df_recovery_trend['amount_scammed'][i]:
            amount_recovered_interim += df_recovery_trend['amount_scammed'][i]
        else:
            amount_recovered_interim += df_recovery_trend['latest_balance_seized'][i]

amount_recovered_list.append(amount_recovered_interim)

print(len(amount_recovered_list))
print(len(dates))
print(dates)
print(amount_recovered_list)

447
447
[datetime.date(2022, 1, 1), datetime.date(2022, 1, 2), datetime.date(2022, 1, 3), datetime.date(2022, 1, 4), datetime.date(2022, 1, 5), datetime.date(2022, 1, 6), datetime.date(2022, 1, 7), datetime.date(2022, 1, 8), datetime.date(2022, 1, 9), datetime.date(2022, 1, 10), datetime.date(2022, 1, 11), datetime.date(2022, 1, 12), datetime.date(2022, 1, 13), datetime.date(2022, 1, 14), datetime.date(2022, 1, 15), datetime.date(2022, 1, 16), datetime.date(2022, 1, 17), datetime.date(2022, 1, 18), datetime.date(2022, 1, 19), datetime.date(2022, 1, 20), datetime.date(2022, 1, 21), datetime.date(2022, 1, 22), datetime.date(2022, 1, 23), datetime.date(2022, 1, 24), datetime.date(2022, 1, 25), datetime.date(2022, 1, 26), datetime.date(2022, 1, 27), datetime.date(2022, 1, 28), datetime.date(2022, 1, 29), datetime.date(2022, 1, 30), datetime.date(2022, 1, 31), datetime.date(2022, 2, 1), datetime.date(2022, 2, 2), datetime.date(2022, 2, 3), datetime.date(2022, 2, 4), datetime.date(2022, 2, 5

In [ ]:
#   x-min & x-max date (Default viewport for graph)                       
    max_date = date.today()
    time = datetime.min.time()
    default_max_datetime = datetime.combine(max_date, time)
    default_max_xview = round(default_max_datetime.replace(tzinfo=timezone.utc).timestamp()) * 1000

    min_date = max_date - timedelta(days = 90)
    default_min_datetime = datetime.combine(min_date, time)
    default_min_xview = round(default_min_datetime.replace(tzinfo=timezone.utc).timestamp()) * 1000
